<img src="pics/8.png" width="35%" />  

0 将自己的用户添加到 docker 用户组：  
sudo usermod -aG docker $USER

0.1 配置国内镜像源（设置里面的Docker Engine）  
{
  "builder": {
    "gc": {
      "defaultKeepStorage": "20GB",
      "enabled": true
    }
  },
  "experimental": false,
  "registry-mirrors": [
    "https://docker.m.daocloud.io",
    "https://huecker.io",
    "https://dockerhub.timeweb.cloud",
    "https://noohub.net"
  ]
}

1 启动 CentOS 7 容器并挂载目录：  
docker run -it --name epymarl_builder -v ~/download2server:/mnt centos:7 /bin/bash

2 安装必要的系统工具：  
修复Yum源（CentOS 7 已经在 2024 年 6 月 30 日停止官方维护，手动将 yum 源指向目前依然可用的“阿里云”镜像站）：  
sed -i s/mirrorlist.centos.org/mirrors.aliyun.com/g /etc/yum.repos.d/CentOS-Base.repo  
sed -i s/#baseurl/baseurl/g /etc/yum.repos.d/CentOS-Base.repo  
sed -i s/mirror.centos.org/mirrors.aliyun.com/g /etc/yum.repos.d/CentOS-Base.repo

修改完源后，清理旧的缓存信息并重新尝试安装工具：
yum clean all  
yum makecache  
yum install -y wget bzip2

3 下载并安装Miniconda  
下载安装脚本：wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh  （得安对应的版本）
执行安装（安装到 /root/miniconda3 目录下）：  
bash Miniconda3-latest-Linux-x86_64.sh -b -p /root/miniconda3  
激活Conda 环境变量：source /root/miniconda3/bin/activate

4 创建虚拟环境  
conda create -n epymarl_env python=3.8.20 -y  
conda activate epymarl_env 

4.1 安装预编译好的二进制包 (Wheel)  
安装缺失的系统工具：yum install -y which gcc-c++  
使用对应的预编译库地址：（执行这一行，pip 会直接去下载已经做好的 linux 二进制包）  
pip install torch-scatter -f https://data.pyg.org/whl/torch-2.1.2+cu121.html

4.2 如果要安装github源代码：  
pip install ./smac-master

4.3 Vmas版本不对：  
把对应的导入改了，再把用到这个导入的地方改掉：  
sed -i 's/from argparse import ArgumentParser, BooleanOptionalAction/from argparse import ArgumentParser/' /root/miniconda3/envs/epymarl_env/lib/python3.8/site-packages/vmas/interactive_rendering.py  
sed -i 's/action=BooleanOptionalAction/action="store_true"/' /root/miniconda3/envs/epymarl_env/lib/python3.8/site-packages/vmas/interactive_rendering.py

5 打包  
先安装：pip install conda-pack  
conda pack -n epymarl_env -o /epymarl_env_centos7_final.tar.gz  
docker cp epymarl_builder:/epymarl_env_centos7_final.tar.gz ~/download2server/  
tar -xzf ../epymarl_env_centos7_final.tar.gz

6 使用  
source /public/software/anaconda/envs/epymarl_env/bin/activate

6.1 加在bin/activate里面
强行屏蔽用户个人目录的干扰，确保 100% 隔离  
export PYTHONNOUSERSITE=1  

6.2 从本地安装库，再到服务器里面去解压  
mkdir -p tensorboard_py38_pkgs  
cd tensorboard_py38_pkgss 
pip download \
    --only-binary=:all: \
    --platform manylinux2014_x86_64 \
    --python-version 3.8 \
    --implementation cp \
    --abi cp38 \
    tensorboard  
下载到服务器里面然后：  
python -m pip install --no-index --find-links=. tensorboard

6.3 查看调库地址优先级：  
python -c "import sys; print('\n'.join(sys.path))"

7 在docker下载库，到服务器里面去安装：  
7.1 打开Docker Desktop 程序  
7.2 在WSL中启动容器：docker start epymarl_builder  
7.3 进入容器内部：docker exec -it epymarl_builder /bin/bash  
7.4 创建文件夹并进入：mkdir /tmp/packs/torchmetrics |||| cd /tmp/packs/torchmetrics  
7.5 激活conda系统，再进入虚拟环境：source /root/miniconda3/bin/activate |||| conda activate epymarl_env  
7.6 下载对应库：pip download torchmetrics -d ./ -i https://pypi.tuna.tsinghua.edu.cn/simple --no-deps  
7.7 顺便在Docker里面安装：pip install --no-index --find-links=./ torchmetrics  
7.8 从Docker拷贝到本地电脑（在本地终端执行！）：docker cp epymarl_builder:/tmp/packs/torchmetrics /mnt/e/packs 
7.9 回到服务器里面对应虚拟环境离线安装：pip install --no-index --find-links=./ torchmetrics  
番外（清空所有下载失败或残留的缓存文件）：pip cache purge